# SQLite 01 - Introduction

<img src="http://sqlite.com/images/sqlite370_banner.gif" >


#### 2016 이승준 fb.com/plusjune

In [1]:
%matplotlib inline
from IPython.display import display, Image

# SQLite 개요 
* http://sqlite.com
* 별도 서버나 프로세스, 특별한 설정을 할 필요도 없는, 트랜잭션이 가능한 SQL 데이타베이스 엔진
* 오픈소스, 어떤 용도로든 무료
* 대부분의 브라우저, 거의 모든 스마트폰, 수 많은 프로그램에 사용

# SQLite 특징
* SQL 데이타베이스 엔진 내장
* 별도의 서버 프로세스가 없이, 직접 읽기/쓰기
* 한 개의 파일에 여러 테이블, 색인, 트리거, 뷰 관리
* 크로스 플랫폼, 작은 라이브러리: 200~350 KB, 매우 작은 메모리 (스택 4KB, Heap 100KB)

# 설치
* 우분투(14.04)에는  기본 포함
* SQLite3 은 파이썬 2.5.x 이상에 기본 포함

```bash
$ sudo apt-get install sqlite3
```

# SqLite 콘솔

```
$ sqlite3 dbname.db3
```

sqlite> 
* .help
* .tables
* .schema [table]
* .exit

# SqLite 콘솔 
```bash
$ sqlite3 news.db3
SQLite version 3.8.2 2013-12-06 14:53:30
Enter ".help" for instructions
Enter SQL statements terminated with a ";"

sqlite> .help
.backup ?DB? FILE      Backup DB (default "main") to FILE
.bail ON|OFF           Stop after hitting an error.  Default OFF
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format

<...중략...>

sqlite> .schema news
CREATE TABLE news (id varchar(20), time datetime, title text, agency varchar(20));

sqlite> .tables
news
sqlite> select * from news;
0003024670|2013-08-29 11:39|국민은행 가족친화 기업에 금리 인하|매일경제
0003024522|2013-08-29 09:03|美양적완화 축소는 은행업종에 긍정적|매일경제
0002464356|2013-08-29 08:00|<진화하는 MTS> KB투자증권, KB스마톡S|헤럴드생생뉴스
0003106995|2013-08-29 07:03|LG상사, STX에너지 인수전에 KB금융 컨소시엄|머니투데이
sqlite> .exit
$
```

# import CSV to Table

```bash
sqlite> .separator ","
sqlite> .import filecsv.csv mytable
```

# Python 에서 SQLite 사용 


In [3]:
import sqlite3

print (sqlite3.version) # pysqlite 버전
print (sqlite3.sqlite_version) # SQLite 데이터베이스 라이브러리 버전

2.6.0
3.8.2


In [4]:
#  파일이 존재하면 삭제 (사용 주의 )
import os

fname = 'test_news.db3'
if os.path.isfile(fname):
    os.unlink(fname)
    
#  파일이 존재하지 않으면 생성 된다.
db = sqlite3.connect(fname)    

# 일반적인 사용 수순
1. import sqlite3 - sqlite3 모듈 임포트 
2. connect() - 연결객체 생성
3. cursor() - 커서객체 생성
4. execute() - SQL문 실행
5. fetch() - 실행결과 가져오기

In [5]:
# 1. import sqlite3 - sqlite3 모듈 임포트 
import sqlite3

con = None

try:
    # 2. connect() - 데이터베이스 파일에 연결(혹은 생성)하고, 연결 객체를 반환
    con = sqlite3.connect('test_news.db3')
    
    # 3. cursor() - 커서객체 생성
    cur = con.cursor()    

    # 4. execute() - SQL문 실행
    cur.execute('SELECT SQLITE_VERSION()')
    
    # 5. fetch() - 실행결과 가져오기
    # 커서는 결과집합(result set)을 순회하여 데이터를 읽어온다.
    val = cur.fetchone()
    print ("SQLite version: %s" % val)

except sqlite3.Error as e:
    # 실패하면, 예외처리 except 구문을 실행
    print ("Error %s:" % e.args[0])
    
finally:    
    if con:
        con.close()

SQLite version: 3.8.2


# with문 사용
* with 문을 사용하여 간편하게 표현
* with 문은 with 블럭의 끝에서 열린 연결이나 열린 파일을 자동으로 close

In [6]:
import sqlite3

con = sqlite3.connect('test_news.db3')

with con:
    cur = con.cursor()    
    cur.execute('SELECT SQLITE_VERSION()')
    
    val = cur.fetchone()
    print ("SQLite version: %s" % val)


SQLite version: 3.8.2


# 테이블 만들기

* create table
* timestamp
* char(n), varchar(n), integer, float, text

In [7]:
con = sqlite3.connect('test_news.db3')

with con:
    cur = con.cursor()
    cur.execute('create table news (id varchar(20), time timestamp, title text, agency varchar(20))')

# 확인
```
sqlite> .schema news
```

# 테이블 삭제

In [8]:
con = sqlite3.connect('test_news.db3')

with con:
    cur = con.cursor()
    cur.execute('drop table news')
    
    # 더 좋은 표현
    cur.execute('drop table if exists news')

# 테이블이 존재하지 않으면 생성

In [9]:
con = sqlite3.connect('test_news.db3')

with con:
    cur = con.cursor()
    cur.execute('create table if not exists news (id varchar(20), time timestamp, title text, agency varchar(20))')


# 데이터 삽입

In [10]:
con = sqlite3.connect('test_news.db3')
   
with con:
    cur = con.cursor()
    cur.execute("insert into news VALUES('0003024670', '2013-08-29 11:39', '국민은행 가족친화 기업에 금리 인하', '매일경제')")
    cur.execute("insert into news VALUES('0003024522', '2013-08-29 09:03', '美양적완화 축소는 은행업종에 긍정적', '매일경제')")
    cur.execute("insert into news VALUES('0002464356', '2013-08-29 08:00', '<진화하는 MTS> KB투자증권, KB스마톡S', '헤럴드생생뉴스')")
    cur.execute("insert into news VALUES('0003106995', '2013-08-29 07:03', 'LG상사, STX에너지 인수전에 KB금융 컨소시엄', '머니투데이')")
    

# 한번에 여러 데이터 삽입

In [11]:
con = sqlite3.connect('test_news.db3')

articles = (
    ('0003024670', '2013-08-29 11:39', u'국민은행 가족친화 기업에 금리 인하', u'매일경제'),
    ('0003024522', '2013-08-29 09:03', u'美양적완화 축소는 은행업종에 긍정적', u'매일경제'),
    ('0002464356', '2013-08-29 08:00', u'<진화하는 MTS> KB투자증권, KB스마톡S', u'헤럴드생생뉴스'),
    ('0003106995', '2013-08-29 07:03', u'LG상사, STX에너지 인수전에 KB금융 컨소시엄', u'머니투데이'),
)

with con:
    cur = con.cursor()
    cur.executemany("insert into news values (?, ?, ?, ?)", articles)

# 커밋, 롤백
* con.commit()
* con.rollback()


# 여러 SQL문 실행

In [12]:
sql_script = '''
    drop table if exists news;
    create table news (id varchar(20), time datetime, title text, agency varchar(20));
    insert into news VALUES('0003024670', '2013-08-29 11:39', '국민은행 가족친화 기업에 금리 인하', '매일경제');
    insert into news VALUES('0003024522', '2013-08-29 09:03', '美양적완화 축소는 은행업종에 긍정적', '매일경제');
    insert into news VALUES('0002464356', '2013-08-29 08:00', '<진화하는 MTS> KB투자증권, KB스마톡S', '헤럴드생생뉴스');
    insert into news VALUES('0003106995', '2013-08-29 07:03', 'LG상사, STX에너지 인수전에 KB금융 컨소시엄', '머니투데이');
'''

try:
    con = sqlite3.connect('test_news.db3')
    cur = con.cursor()
    cur.executescript(sql_script)
    con.commit()
except sqlite3.Error as e:
    if con:
        con.rollback()
    print ("error %s" % e.args[0])

finally:
    if con:
        con.close()
    

# 데이터 읽기

In [13]:
con = sqlite3.connect('test_news.db3')

with con:
    cur = con.cursor() 
    cur.execute('select * from news')
    
    rows = cur.fetchall()
    for row in rows:
        print ("%s | %s(%s) " %  (row[2], row[1], row[3]) )

국민은행 가족친화 기업에 금리 인하 | 2013-08-29 11:39(매일경제) 
美양적완화 축소는 은행업종에 긍정적 | 2013-08-29 09:03(매일경제) 
<진화하는 MTS> KB투자증권, KB스마톡S | 2013-08-29 08:00(헤럴드생생뉴스) 
LG상사, STX에너지 인수전에 KB금융 컨소시엄 | 2013-08-29 07:03(머니투데이) 


# 조건에 해당하는 row를 하나씩 처리

In [14]:
con = sqlite3.connect('test_news.db3')

with con:
    cur = con.cursor() 
    cur.execute('select * from news where agency = "매일경제"')

    while True:
        row = cur.fetchone()
        if row == None:
            break
        print ("%s | %s | %s" % (row[2], row[1], row[3]))
        

국민은행 가족친화 기업에 금리 인하 | 2013-08-29 11:39 | 매일경제
美양적완화 축소는 은행업종에 긍정적 | 2013-08-29 09:03 | 매일경제
